In [1]:
# from dem_stitcher.stitcher import stitch_dem
# import rasterio
# from rasterio import plot
# import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import asf_search as asf
from datetime import datetime, date, timedelta
from typing import List

In [2]:
# for formatting the datetime object to asfsearch syntax
def datetime2asfsearch(entered_date: datetime) -> str:
    return datetime.strftime(entered_date,'%Y') + '-' + datetime.strftime(entered_date,'%m') + '-' + datetime.strftime(entered_date,'%d') + 'T' + datetime.strftime(entered_date,'%H') + ':' + datetime.strftime(entered_date,'%M') + ':' + datetime.strftime(entered_date,'%S') + 'Z'

In [3]:
def get_coverage(sensor: List[str], aoi: Point, date: List[datetime] = None) -> List[dict]:
    """
    Sensor: choose Sentinel-1
    AOI: enter coordinates as tuple
    date: leave as none if searching today, else enter time range as datetime tuple: datetime(YYYY,MM,DD)
    """
    freq = {}
    next_acq = {}
    area = {}
    
    for sensor_name in sensor:
        freq[sensor_name] = ''
        next_acq[sensor_name] = ''
        area[sensor_name] = ''
        
        if date == None:
            today = date.today()
            start = str(today) + 'T00:00:00Z'
            end = str(today) + 'T23:59:59Z'
        else:
            start = datetime2asfsearch(date[0])
            end = datetime2asfsearch(date[1])

        opts = {
            'platform': asf.PLATFORM.SENTINEL1,
            'maxResults': 600,
            'start': start,
            'end': end
        }
        results = asf.search(**opts)

        coords = []
        for i in range(len(results)):
            coords.append(results[i].geometry['coordinates'][0])

        # find all intersecting polygons and save as list
        counter = 0
        polygon_list = []
        for i in range(len(results)):
            poly = Polygon(coords[i])
            if aoi.within(poly):
                counter += 1
                polygon_list.append(Polygon(coords[i]))
        
        # calculate frequency
        if counter == 0:
            freq[sensor_name] = 'There is no coverage during this time'
            area[sensor_name] = 0
        else:
            delta = date[1].date() - date[0].date()
            freq[sensor_name] = str(delta / counter)
        
        # find next acquisition time, if search time is today then returns 'N/A'
        if date == None:
            next_acq[sensor_name] = 'N/A'
        else:
            
            acq = False
            rep = 0
            # put arbitrary hard stop at 10 searches (1000 days)
            while acq == False or rep > 10:
                
                start = datetime2asfsearch(date[1] + timedelta(days = 100 * rep))
                end = datetime2asfsearch(date[1] + timedelta(days = 100 * (rep + 1)))
                opts = {
                    'platform': asf.PLATFORM.SENTINEL1,
                    'maxResults': 100,
                    'start': start,
                    'end': end
                }
                
                results = asf.search(**opts)

                coords = []
                for i in range(len(results)):
                    coords.append(results[i].geometry['coordinates'][0])

                for i in range(len(results)):
                    poly = Polygon(coords[i])
                    if aoi.within(poly):
                        acq = True
                        break
                        
                if acq:
                    next_acq[sensor_name] = results[i].properties['startTime']
                    
                rep += 1
            if next_acq[sensor_name] == '':
                next_acq[sensor_name] = 'Search yielded no results'
                
        if counter != 0:
            
            area[sensor_name] = polygon_list[0]
            if counter != 1:
                
                for i in range(len(polygon_list) - 1):
                    area[sensor_name] = area[sensor_name].intersection(polygon_list[i + 1])
            
    return freq, next_acq, area

In [4]:
freq, next_acq, area = get_coverage(['Sentinel1'],Point(105, 74),[datetime(2015,1,1), datetime(2017,1,1)])

In [5]:
print(freq['Sentinel1'])
print(next_acq['Sentinel1'])

182 days, 18:00:00
2017-07-19T23:41:48.000000


In [6]:
print(area['Sentinel1'])

POLYGON ((110.185722 72.48864, 98.24223869384016 73.51737254950272, 100.95087756050029 76.86508249644316, 115.437492 75.639885, 110.185722 72.48864))
